In [1]:
import pickle
import yaml
import pandas as pd
import numpy as np

from tqdm import tqdm_notebook, tqdm

from skopt import forest_minimize
from skopt.plots import plot_convergence
from skopt.utils import use_named_args

from sklearn import metrics
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import f1_score, make_scorer


import warnings
from multiprocessing import cpu_count

from skopt.utils import use_named_args
from skopt.space import Real, Categorical, Integer
from skopt import gp_minimize,gbrt_minimize
from src.functions import (
    generate_stratified_folds,
    cross_validate_score,
    generate_space_dimension,
    update_parameter_yaml
)

from xtlearn.utils import load_pickle, dump_pickle
from copy import deepcopy

warnings.filterwarnings("ignore")
pd.options.display.max_columns = None

In [2]:
n_jobs = cpu_count()
inp_path = "data/"
sub_path = "submissions/"
%run src/import_parameters.py
TARGET = pd.read_csv(sub_path + "68200.csv")["predicted"]

In [3]:
from lightgbm  import LGBMClassifier

In [4]:
estimator = LGBMClassifier()


### Reading data from Porto Seguro Data Challenge 2021
* Remover id

In [5]:
# df_teste = pd.DataFrame(list_results,columns = ["feature", "full", "without", "only"])
# df_teste["diff"] = df_teste["without"] - df_teste["full"]
# df_teste = df_teste.sort_values("diff")

# def func(x):
#     if x["without"] <= x["full"]:
#         return "drop"
    
# df_teste = df_teste.assign(action=df_teste.apply(func,axis=1))

# df_teste.to_csv("data/f1_score_for_features.csv",index=False)

In [6]:
metadata = pd.read_csv(inp_path + 'metadata.csv')
test = pd.read_csv(inp_path + 'test.csv')
train = pd.read_csv(inp_path + 'train.csv')
train = train.sample(frac=1, random_state=0).reset_index(drop=True)  # shuffling dataset
sub = pd.read_csv(inp_path + 'submission_sample.csv')
feature_selection = pd.read_csv("data/f1_score_for_features.csv")

###########################

# Excluindo colunas por terem muitos missings
train = train.drop(columns = ["var60","var65","var66"],errors="ignore")
test = test.drop(columns = ["var60","var65","var66"],errors="ignore")


cols_to_drop = feature_selection[feature_selection["action"]=="drop"]["feature"].to_list()
train = train.drop(columns = cols_to_drop,errors="ignore")
test = test.drop(columns = cols_to_drop,errors="ignore")

# train = train.replace(-999,np.nan)
# nans_count = train.isna().mean().to_dict()

# cols_to_flag = []

# for k in dict(filter(lambda x: x[1] > 0.10,nans_count.items())):
#     if nans_count[k] > 0:
#         #cols_to_flag.append(k)
#         train[k+"_nan_flag"] = train[k].isna().astype(int)
#         test[k+"_nan_flag"] = test[k].isna().astype(int)

## Função de Treino

In [16]:
def score_function(cv, corr):
#     std_cv = 0.012
#     mean_cv = -0.71
#     std_corr = 0.06
#     mean_corr = 0.99
    std_cv = 0.012
    mean_cv = -0.69
    std_corr = 0.06
    mean_corr = 0.99

    score_cv = cv  #* np.exp(-0.5 * (cv - mean_cv) ** 2 / std_cv ** 2)
    score_corr = corr * np.exp(-0.5 * (corr - mean_corr) ** 2 / std_corr ** 2)
    
    #print("cv =",cv,"  |  corr =",corr)

    return np.sqrt(0.15 * (score_cv / mean_cv - 1) ** 2 + 0.85 * (score_corr / mean_corr - 1) ** 2)

In [17]:
def predict(model, X_test, threshold=0.5):
    predict_prob = model.predict_proba(X_test)[:, 1]
    condition = lambda x: x > threshold
    vec_condition = np.vectorize(condition)
    return np.where(vec_condition(predict_prob), 1.0, 0.0)

In [18]:
def cross_validate_score(
    X,
    y,
    estimator,
    n_folds=5,
    scoring=f1_score,
    threshold=0.3,
    random_state=42,
    verbose=0,
    fit_params={},
):

    scores = []

    temp = X.assign(y=y)

    temp["fold"] = generate_stratified_folds(temp, n_folds=n_folds)

    iterator = (
        range(n_folds) if verbose < 1 else tqdm(range(n_folds), desc="Cross validation")
    )

    for fold in iterator:

        # Separando os dados de treinamento para essa fold
        train_data = temp[temp["fold"] != fold].copy()

        # Separando os dados de teste para esse fold
        test_data = temp[temp["fold"] == fold].copy()

        cv = train_scoring(
            estimator, train_data, test_data, scoring, threshold=threshold, fit_params=fit_params
        )

        scores.append(cv)

    cv = np.mean(scores)

    return -cv

In [19]:
def train_scoring(estimator,train_data,test_data, scoring, threshold=0.5,fit_params = {}):
        
        opt_drop = dict(columns=["fold", "y","id"],errors='ignore')
        
        X_train = train_data.drop(**opt_drop).values

        X_test = test_data.drop(**opt_drop).values

        y_train = train_data["y"].values

        y_test = test_data["y"].values
        
        fit_params["eval_set"] = [(X_test, y_test)]

        estimator.fit(X_train, y_train, **fit_params)
        
        prob_test = estimator.predict_proba(X_test)[:, -1]
        
        y_pred = prob_test > threshold
        
        cv = scoring(y_test, y_pred)

        return cv

    

## Função de Escoragem

In [20]:
def S(par, limits):

    par_scaled = deepcopy(par)

    params = {
        k: (par_scaled[k] * (limits[k][1] - limits[k][0]) + limits[k][0])
        for i, k in enumerate(par_scaled)
    }

    threshold = THRESHOLD
    if "threshold" in params:
        threshold = params.pop("threshold")

    #estimator = XGBClassifier(
    #    n_jobs=n_jobs, random_state=RANDOM_STATE, **params, **DISC_PAR
    #)
    
    estimator = LGBMClassifier(
        n_jobs=n_jobs, random_state=RANDOM_STATE, **params, **DISC_PAR
    )

    fit_params = {
        "early_stopping_rounds": EARLY_STOPPING_ROUNDS,
        "eval_metric": EVAL_METRIC,
        "verbose": VERBOSE,
    }

    cv = cross_validate_score(
        X=train.drop(["y", "id"], 1,errors="ignore"),
        y=train["y"],
        estimator=estimator,
        fit_params=fit_params,
        n_folds=N_FOLDS,
        scoring=SCORING,
        threshold=threshold,
        random_state=RANDOM_STATE,
        verbose=0,
    )

    # Separando os dados de treinamento para essa fold
    train_data = train.copy()

    # Separando os dados de teste para esse fold
    test_data = test.copy()

    X_train = train_data.drop(columns=["fold", "y", "id"], errors="ignore").values

    X_test = test_data.drop(columns=["fold", "y", "id"], errors="ignore").values

    y_train = train_data["y"].values

    y_test = TARGET.values

    fit_params.pop("eval_set")
    fit_params["eval_set"] = [(X_test, y_test)]

    estimator.fit(X_train, y_train, **fit_params)

    y_pred = predict(estimator, X_test, threshold=threshold).astype(int)

    corr = pd.DataFrame({0: TARGET, 1: y_pred}).corr()[0][1]

    if pd.isnull(corr):
        corr = 0

    score = score_function(cv, corr)
    # score = 0.5 * (cv / 0.71 + 1) ** 2 + 0.5 * (corr / 0.99 - 1) ** 2

    return score

## Otimizadores

In [21]:
def local_opt(par, n_iter=20, epsilon=0.1, alpha=0.1, alpha_end=None, n_tries=4):

    par.update({"threshold": THRESHOLD})
    limits.update({"threshold": [0.1, 0.9]})

    par_scaled = {
        k: (par[k] - limits[k][0]) / (limits[k][1] - limits[k][0])
        for i, k in enumerate(par)
    }

    eval_ = []

    S0 = S(par_scaled, limits)

    print("Input Score:", S0)

    learn_rate = deepcopy(alpha)

    cont = 0
    results = [[0,par,S0]]
    for j in range(n_iter):

        print("\n\n###############################################")
        print("Iteração:", j," | Score atual:",S0)

        # Gradiente
        grad = np.zeros(len(par_scaled))
        for dim, k in enumerate(par):
            new_par = {
                k: np.clip(par_scaled[k] + int(i == dim) * epsilon, 0, 1)
                for i, k in enumerate(par_scaled)
            }
            new_S = S(new_par, limits)
            grad[dim] = (new_S - S0) / epsilon

        grad = grad / np.linalg.norm(grad)


        # Atualizando learning_rate
        if alpha_end is not None:
            learn_rate = alpha * (alpha_end / alpha) ** (j / n_iter)

        old_S = S0
        tries_results = [(par_scaled,S0)]
        
        for ind in range(n_tries):

            # Se n_tries for 2, então, primeiramente o fator será 1 e depois 0.5
            # Se n_tries for 3, então, primeiramente o fator será 1 0.66 e 0.33
            
            factor = 1 - ind/n_tries

            new_par_scaled = {
                k: np.clip(par_scaled[k] - factor * learn_rate * grad[i], 0, 1)
                for i, k in enumerate(par_scaled)
            }

            new_S = S(new_par_scaled, limits)
            tries_results.append((new_par_scaled,new_S))

            print("\t\t>>> New S:", new_S)

            if new_S <= S0:
                break
                
            elif new_S == old_S:
                new_par = {
                    k: (new_par_scaled[k] * (limits[k][1] - limits[k][0]) + limits[k][0])
                    for i, k in enumerate(new_par_scaled)
                }
                print("\t\t>>> Converged | Parameters:", new_par)
                break
            
            old_S = new_S
            
        indmin = np.argmin([tup[1] for tup in tries_results])
        new_par_scaled,new_S = tries_results[indmin]

        if new_S < S0:
            S0 = new_S
            par_scaled = new_par_scaled
        else:
            print("Random!")
            prop_par_scaled = {
                k: np.clip(
                    par_scaled[k]
                    - np.random.uniform(
                        -factor * learn_rate * grad[i], factor * learn_rate * grad[i]
                    ),
                    0,
                    1,
                )
                for i, k in enumerate(par_scaled)
            }
            prop_S = S(prop_par_scaled, limits)
            P0 = np.exp(S0)
            new_P = np.exp(prop_S)
            A = min(1, new_P / P0)
            u = np.random.random()
            if u <= A:
                if prop_S < new_S:
                    S0 = prop_S
                    par_scaled = prop_par_scaled
                else:
                    S0 = new_S
                    par_scaled = new_par_scaled
                    
                
        

        par = {
            k: (par_scaled[k] * (limits[k][1] - limits[k][0]) + limits[k][0])
            for i, k in enumerate(par_scaled)
        }
    
        results.append([j+1,par,S0])
        print("iter =", j, "S0 =", S0, " | alpha =", learn_rate)
        print("parameters:",par)

        eval_.append([j, par_scaled, S0])

    eval_.append([n_iter, par_scaled, S(par_scaled, limits)])

    ind = (
        pd.DataFrame({0: np.array(eval_)[:, 0], 1: np.array(eval_)[:, 2]})
        .sort_values(1)
        .iloc[0][0]
    )

    par_scaled = eval_[ind][1]

    params = {
        k: (par_scaled[k] * (limits[k][1] - limits[k][0]) + limits[k][0])
        for i, k in enumerate(par_scaled)
    }
    
    return params, eval_[ind][2], results

In [22]:
def opt(par,n_iter=20,epsilon = 0.1,alpha = 0.1):

    par.update({"threshold": THRESHOLD})
    limits.update({"threshold": [0.1, 0.9]})

    par_scaled = {
        k: (par[k] - limits[k][0]) / (limits[k][1] - limits[k][0])
        for i, k in enumerate(par)
    }

    eval_ = []

    S0 = S(par_scaled, limits)

    grad = np.zeros(len(par_scaled))

    learn_rate = deepcopy(alpha)

    for j in range(n_iter):

        new_par_scaled = {
            k: np.clip(par_scaled[k] - learn_rate * grad[i], 0, 1)
            for i, k in enumerate(par_scaled)
        }

        new_S = S(new_par_scaled, limits)

        # apagar
        S0 = new_S
        par_scaled = new_par_scaled

        if new_S < S0:
            S0 = new_S
            par_scaled = new_par_scaled
        else:
            new_par_scaled = {
                k: np.clip(par_scaled[k] - np.random.uniform(-learn_rate,learn_rate), 0, 1)
                for i, k in enumerate(par_scaled)
            }
            new_S = S(new_par_scaled, limits)
            P0 = np.exp(S0)
            new_P = np.exp(new_S)
            A = min(1,new_P / P0)
            u = np.random.random()
            if u <= A:
                S0 = new_S
                par_scaled = new_par_scaled

        print(S0)

        grad = np.zeros(len(par_scaled))
        eval_.append([j, par_scaled, S0])
        for dim, k in enumerate(par):
            new_par = {
                k: np.clip(par_scaled[k] + int(i == dim) * epsilon, 0, 1)
                for i, k in enumerate(par_scaled)
            }
            new_S = S(new_par, limits)
            grad[dim] = (new_S - S0) / epsilon

        grad = grad / np.linalg.norm(grad)


    eval_.append([n_iter, par_scaled, S(par_scaled, limits)])
    
    ind = (
        pd.DataFrame({0: np.array(eval_)[:, 0], 1: np.array(eval_)[:, 2]})
        .sort_values(1)
        .iloc[0][0]
    )

    par_scaled = eval_[ind][1]
    
    params = {
        k: (par_scaled[k] * (limits[k][1] - limits[k][0]) + limits[k][0])
        for i, k in enumerate(par_scaled)
    }
    
    return params, eval_[ind][2]



## Importando Parâmetros

In [23]:
from xtlearn.utils import load_pickle, dump_pickle

%run src/import_parameters.py

# Parameters space
parameters = yaml.safe_load(open("data/parameters.yaml", "r"))
hyperparameters = yaml.safe_load(open("data/hyperparameters.yaml", "r"))
space = [generate_space_dimension(x) for x in parameters]
PARAMETER_NAMES = [elem.name for elem in space]

params_real = list(filter(lambda x: x['type']=='real',parameters))
params_disc = list(filter(lambda x: x['type']!='real',parameters))

params = {x['parameter']:x['estimate']for x in params_real}
limits = {x['parameter']:x['range']for x in params_real}
steps = {x['parameter']:x['step']for x in params_real}

DISC_PAR = {
    'n_estimators': 200, 
    'max_depth': 6, 
    #'num_parallel_tree': 2,
    'max_delta_step': 0
}
space = [generate_space_dimension(x) for x in params_real]
PARAMETER_NAMES = [elem.name for elem in space]

# x0 = list(params.values())
# score = 2

# dump_pickle(space,"data/best_space.pkl")
# dump_pickle(PARAMETER_NAMES,"data/best_PARAMETER_NAMES.pkl")
# dump_pickle(x0,"data/best_x0.pkl")
# dump_pickle(hyperparameters['THRESHOLD'],'data/best_threshold.pkl')
# dump_pickle(score,"data/best_y0.pkl")

## Testes Unidimensionais

In [24]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler
# from xtlearn.preprocessing import MeanModeImputer

## Otimização Local

In [ ]:
%%time

PARAMETER_NAMES = load_pickle("data/best_PARAMETER_NAMES.pkl")
THRESHOLD = load_pickle("data/best_threshold.pkl")
x0 = load_pickle("data/best_x0.pkl")
score = load_pickle("data/best_y0.pkl")
params = dict(zip(PARAMETER_NAMES, x0))

list_mcmc = [[params,score]]
for i in range(10):
    try:
        params_temp, score_temp, result_temp = local_opt(
            params,
            n_iter=10,
            epsilon= 0.1 * 10 ** np.random.uniform(-4, 0),
            alpha= 10 ** np.random.uniform(-3, -1),
            alpha_end = 10 ** np.random.uniform(-3,-1),
            n_tries=50,
        )

        indmin = np.argmin([tup[2] for tup in result_temp])
        *_, params_temp, score_temp = result_temp[indmin]

        if score > score_temp:
            score = score_temp
            params = params_temp
            result = result_temp
            list_mcmc.append([params, score])

        else:
            A = np.exp(-score_temp) / np.exp(-score)

            u = np.random.random()

            if A > u:
                score = score_temp
                params = params_temp
                result = result_temp
                list_mcmc.append([params, score])
    except Exception as err:
        print(err)

indmin = np.argmin([tup[1] for tup in list_mcmc])
params, score = list_mcmc[indmin]

x0 = [x[1] for x in filter(lambda x: x[0] != "threshold",params.items())]
dump_pickle(x0,"data/best_x0.pkl")
dump_pickle(score,"data/best_y0.pkl")
THRESHOLD = params["threshold"]
dump_pickle(THRESHOLD,"data/best_threshold.pkl")

Input Score: 0.32241171621643755


###############################################
Iteração: 0  | Score atual: 0.32241171621643755
		>>> New S: 0.3809401119294088
		>>> New S: 0.3811915204110212
		>>> New S: 0.3773272372262941
		>>> New S: 0.38094494445616534
		>>> New S: 0.370571280689606
		>>> New S: 0.4103336773263072
		>>> New S: 0.4143034066052334
		>>> New S: 0.37283424425447637
		>>> New S: 0.39299762716203035
		>>> New S: 0.3645874633519659
		>>> New S: 0.33043066946608973
		>>> New S: 0.38542529661903707
		>>> New S: 0.36239609384438265
		>>> New S: 0.3779208298722645
		>>> New S: 0.385858329905516
		>>> New S: 0.36572881234642546
		>>> New S: 0.37587464054297004
		>>> New S: 0.3682519905639584
		>>> New S: 0.3747112947589876
		>>> New S: 0.3573895768336538
		>>> New S: 0.3660832831483409
		>>> New S: 0.35847258112438646
		>>> New S: 0.36383481096229187
		>>> New S: 0.35270293200621583
		>>> New S: 0.3807704278485276
		>>> New S: 0.3661817631010007
		>>> New S: 0.3568487342598

# MCMC

In [35]:
score

0.543

In [46]:
%%time
from tqdm import tqdm

PARAMETER_NAMES = load_pickle("data/best_PARAMETER_NAMES.pkl")
THRESHOLD = load_pickle("data/best_threshold.pkl")
x0 = load_pickle("data/best_x0.pkl")
score = load_pickle("data/best_y0.pkl")
params = dict(zip(PARAMETER_NAMES, x0))


def f(params, limits, index):
    par_scaled = {
        k: (params[k] - limits[k][0]) / (limits[k][1] - limits[k][0])
        for i, k in enumerate(params)
    }
    s = S(par_scaled, limits)
    result = np.exp(-s)

    print("\n#################################################")
    print(f"{index} - score = {s}  |  prob = {result}")
    print(params)

    return result


params = dict(zip(PARAMETER_NAMES, x0))
P = f(params, limits, 0)

params_max = params
P_max = P

list_mcmc = [(0, params, P, P_max)]
for ind in range(8400):

    new_params = {
        k: np.clip(
            params[k] + 0.333 * np.random.randn() * steps[k], limits[k][0], limits[k][1]
        )
        for k in params
    }

    new_P = f(new_params, limits, ind + 1)

    A = min(1, new_P / P)
    u = np.random.random()

    if u < A:
        P = new_P
        params = new_params

    if P > P_max:
        P_max = P
        params_max = params

    list_mcmc.append((ind, params, P, P_max))

indmin = np.argmax([tup[2] for tup in list_mcmc])
*_,params, pmax = list_mcmc[indmin]
score = -np.log(pmax)

x0 = [x[1] for x in filter(lambda x: x[0] != "threshold", params_max.items())]
# THRESHOLD = params_max["threshold"]
dump_pickle(x0, "data/best_x0.pkl")
dump_pickle(score, "data/best_y0.pkl")
dump_pickle(THRESHOLD, "data/best_threshold.pkl")


#################################################
0 - score = 0.5477882322934622  |  prob = 0.578227301575483
{'learning_rate': 0.07234854009556661, 'min_child_weight': 0.10311050258987084, 'subsample': 0.39703575514097145, 'reg_alpha': 0.8518533874150784, 'reg_lambda': 0.6058731277317677, 'colsample_bynode': 0.5296917410366301, 'colsample_bytree': 0.05170479667699425, 'path_smooth': 0.2551823769930473}

#################################################
1 - score = 0.9224230323026632  |  prob = 0.39755458550012446
{'learning_rate': 0.07271492540241592, 'min_child_weight': 0.1, 'subsample': 0.3962510361339386, 'reg_alpha': 0.8520266690935441, 'reg_lambda': 0.6091166783153117, 'colsample_bynode': 0.5296189159752714, 'colsample_bytree': 0.052352919789940576, 'path_smooth': 0.2674168504581467}

#################################################
2 - score = 0.5527034525213611  |  prob = 0.5753921604259571
{'learning_rate': 0.07253697111635929, 'min_child_weight': 0.12652603164694293, 'subsa


#################################################
21 - score = 0.5481155623666407  |  prob = 0.5780380613642402
{'learning_rate': 0.07647139119672208, 'min_child_weight': 0.1914299668399015, 'subsample': 0.3973299810760653, 'reg_alpha': 0.8566501996681747, 'reg_lambda': 0.6065965543057926, 'colsample_bynode': 0.5308559030378309, 'colsample_bytree': 0.05188320839722794, 'path_smooth': 0.03960954403770873}

#################################################
22 - score = 0.6609820972412073  |  prob = 0.5163439853958331
{'learning_rate': 0.07690165802866834, 'min_child_weight': 0.19737719961762973, 'subsample': 0.3965193403429212, 'reg_alpha': 0.8589781276510153, 'reg_lambda': 0.6069506873338285, 'colsample_bynode': 0.531210597755804, 'colsample_bytree': 0.05217237550253716, 'path_smooth': 0.0}

#################################################
23 - score = 0.5839492512567085  |  prob = 0.5576915477468466
{'learning_rate': 0.07707843974872877, 'min_child_weight': 0.17737030822457805, 'subs


#################################################
42 - score = 0.9717985711115755  |  prob = 0.3784018430746585
{'learning_rate': 0.07664683563002474, 'min_child_weight': 0.21826612435203077, 'subsample': 0.39826254048817816, 'reg_alpha': 0.8467312450246339, 'reg_lambda': 0.6118959709637612, 'colsample_bynode': 0.5304830355624631, 'colsample_bytree': 0.052370591493924507, 'path_smooth': 0.18423465814102935}

#################################################
43 - score = 0.5493596001344441  |  prob = 0.5773194072937623
{'learning_rate': 0.0765661430360258, 'min_child_weight': 0.23968182303192004, 'subsample': 0.3996377675249306, 'reg_alpha': 0.8465745383791744, 'reg_lambda': 0.6119516469823366, 'colsample_bynode': 0.5308198496210076, 'colsample_bytree': 0.05228776466006677, 'path_smooth': 0.16558567186430126}

#################################################
44 - score = 0.8890652759690161  |  prob = 0.4110397820056299
{'learning_rate': 0.07627838047303583, 'min_child_weight': 0.21602

KeyboardInterrupt: 

In [76]:
score

0.32253947101795366

## Otimização Skopt

In [29]:
@use_named_args(space)
def train_function_test(**params):
    print(f"Testing parameters: {params.values()}")

    try:
#         estimator = XGBClassifier(
#             n_jobs=n_jobs,
#             random_state=RANDOM_STATE,
#             **params,
#             **DISC_PAR,
#         )
        estimator = LGBMClassifier(
            n_jobs=n_jobs,
            random_state=RANDOM_STATE,
            **params,
            **DISC_PAR,
        )

        fit_params = {
            "early_stopping_rounds": EARLY_STOPPING_ROUNDS,
            "eval_metric": EVAL_METRIC,
            "verbose": VERBOSE,
        }

        cv = cross_validate_score(
            X=train.drop(["y","id"], 1,errors="ignore"),
            y=train["y"],
            estimator=estimator,
            fit_params=fit_params,
            n_folds=N_FOLDS,
            scoring=SCORING,
            threshold=THRESHOLD,
            random_state=RANDOM_STATE,
            verbose=0,
        )
        
        
        fit_params["eval_set"] = [(test.drop("id",1,errors="ignore"), TARGET)]

        estimator.fit(train.drop(["y","id"], 1,errors="ignore"), train["y"])

        y_pred = predict(estimator, test.drop("id",1,errors="ignore"), threshold=THRESHOLD).astype(int)

        corr = pd.DataFrame({0: TARGET, 1: y_pred}).corr()[0][1]
        
        print(cv,corr)

        # corr = 1 - np.abs(TARGET-y_pred).mean()

        # score = 0.5 * (cv / 0.71 + 1) ** 2 + 0.5 * (corr / 0.99 - 1) ** 2
        score = score_function(cv, corr)

        if pd.isnull(score):
            score = 1
        else:

            pd.DataFrame([0] + list(params.values()) + [score]).T.to_csv(
                "data/optimization.csv", index=False, mode="a", header=False
            )

        print(f"\n\t<<<<< cv = {cv}   |   corr = {corr} |  score = {score}>>>>>")
        print(f"\n\t{params}\n")

        return score

    except Exception as err:
        print(err)
        return 1

In [35]:
%%time
N_CALLS = 1000
N_RANDOM_STARTS = 10

# results = pd.read_csv("data/optimization.csv").sort_values('score')
# x0 = list(results.drop(["iteration", "score"], 1).iloc[0].values)
# y0 = results["score"].iloc[0].

space = load_pickle("data/best_space.pkl")
PARAMETER_NAMES = load_pickle("data/best_PARAMETER_NAMES.pkl")
x0 = load_pickle("data/best_x0.pkl")
score = load_pickle("data/best_y0.pkl")


res_gp = gbrt_minimize(
# res_gp = gp_minimize(
    #train_function,
    train_function_test,
    space,
    x0=x0,
    y0=score,
    random_state=None,
    verbose=1,
    n_calls=N_CALLS,
    n_random_starts=N_RANDOM_STARTS,
    n_jobs=n_jobs,
)

x0 = res_gp.x
score = res_gp.fun

dump_pickle(x0,"data/best_x0.pkl")
dump_pickle(score,"data/best_y0.pkl")

# xgb_best_params = res_gp.x

# print(f"Best found params: {xgb_best_params}")

# plot_convergence(res_gp)

# results = (
#     pd.DataFrame(res_gp.x_iters, columns=PARAMETER_NAMES)
#     .assign(score=res_gp.func_vals)
#     .reset_index()
#     .rename(columns={"index": "iteration"})
#     .sort_values("score")
# )

# results.to_csv("data/optimization.csv", index=False, mode='a',header=False)

#update_parameter_yaml(results,parameters)

# display(results)

Iteration No: 1 started. Evaluating function at random point.
Iteration No: 1 ended. Evaluation done at random point.
Time taken: 0.0003
Function value obtained: 0.3838
Current minimum: 0.3838
Iteration No: 2 started. Evaluating function at random point.
Testing parameters: dict_values([0.08905878743324946, 1.2339788706020118, 0.9032816338143139, 0.5926383736954801, 0.4504875148360157, 0.9096133018332754, 0.44582040286144375, 7.359313264688845])
-0.6690432527162944 0.8852047269453519

	<<<<< cv = -0.6690432527162944   |   corr = 0.8852047269453519 |  score = 0.7427001161761554>>>>>

	{'learning_rate': 0.08905878743324946, 'min_child_weight': 1.2339788706020118, 'subsample': 0.9032816338143139, 'reg_alpha': 0.5926383736954801, 'reg_lambda': 0.4504875148360157, 'colsample_bynode': 0.9096133018332754, 'colsample_bytree': 0.44582040286144375, 'path_smooth': 7.359313264688845}

Iteration No: 2 ended. Evaluation done at random point.
Time taken: 1.2632
Function value obtained: 0.7427
Current

-0.6601058347562571 0.9067205008984741

	<<<<< cv = -0.6601058347562571   |   corr = 0.9067205008984741 |  score = 0.5999272074254924>>>>>

	{'learning_rate': 0.020491949163269192, 'min_child_weight': 0.17807470409961068, 'subsample': 0.09266075526933638, 'reg_alpha': 0.09774275856812023, 'reg_lambda': 0.060055218847217, 'colsample_bynode': 0.14204313856451592, 'colsample_bytree': 0.5684657291532433, 'path_smooth': 2.5519035091555327}

Iteration No: 12 ended. Search finished for the next optimal point.
Time taken: 2.1749
Function value obtained: 0.5999
Current minimum: 0.3838
Iteration No: 13 started. Searching for the next optimal point.
Testing parameters: dict_values([0.02587682140513212, 0.11669466356682513, 0.42608845436314147, 0.03024867371180851, 0.3658777984245016, 0.02755090358835722, 0.9438911888963192, 1.562939135438768])
-0.6228555124390138 0.8163311370354378

	<<<<< cv = -0.6228555124390138   |   corr = 0.8163311370354378 |  score = 0.9112082436079558>>>>>

	{'learning_rat

-0.6572476237975722 0.9024711472373322

	<<<<< cv = -0.6572476237975722   |   corr = 0.9024711472373322 |  score = 0.6322283950797937>>>>>

	{'learning_rate': 0.00730163789970275, 'min_child_weight': 0.9219674349244984, 'subsample': 0.02123365268124922, 'reg_alpha': 0.01906278414091467, 'reg_lambda': 0.7572835106034028, 'colsample_bynode': 0.5078426570626545, 'colsample_bytree': 0.9488510622008905, 'path_smooth': 9.973999443040633}

Iteration No: 22 ended. Search finished for the next optimal point.
Time taken: 1.8615
Function value obtained: 0.6322
Current minimum: 0.3838
Iteration No: 23 started. Searching for the next optimal point.
Testing parameters: dict_values([0.03229071133533678, 0.17344742226173634, 0.23493366455023285, 0.8695892055223008, 0.48773217842025995, 0.3874061822549568, 0.7878246604787555, 8.64011442149129])
-0.6674458815194632 0.9263654474419828

	<<<<< cv = -0.6674458815194632   |   corr = 0.9263654474419828 |  score = 0.43054861509897974>>>>>

	{'learning_rate': 

-0.5341156214214146 0.7170459015788196

	<<<<< cv = -0.5341156214214146   |   corr = 0.7170459015788196 |  score = 0.9260758368652792>>>>>

	{'learning_rate': 0.009600745549777155, 'min_child_weight': 0.11641742988922311, 'subsample': 0.99378743804645, 'reg_alpha': 0.30423610011161534, 'reg_lambda': 0.4680999582826679, 'colsample_bynode': 0.2633997524966118, 'colsample_bytree': 0.16213398064484455, 'path_smooth': 3.8742441141021553}

Iteration No: 32 ended. Search finished for the next optimal point.
Time taken: 1.2438
Function value obtained: 0.9261
Current minimum: 0.3838
Iteration No: 33 started. Searching for the next optimal point.
Testing parameters: dict_values([0.01513577649860165, 0.16662566912336113, 0.133431880950472, 0.7457088569678677, 0.23163204833270984, 0.0038389681380801574, 0.9256496179984678, 9.892696856688202])
-0.569083922872687 0.7538324248905709

	<<<<< cv = -0.569083922872687   |   corr = 0.7538324248905709 |  score = 0.9241466216976926>>>>>

	{'learning_rate': 

-0.6651783175664789 0.8867350749520524

	<<<<< cv = -0.6651783175664789   |   corr = 0.8867350749520524 |  score = 0.7343058252255351>>>>>

	{'learning_rate': 0.09294790534733671, 'min_child_weight': 0.13985501448440307, 'subsample': 0.2645604471150898, 'reg_alpha': 0.3154786613853722, 'reg_lambda': 0.15440246023189344, 'colsample_bynode': 0.31253830709450064, 'colsample_bytree': 0.8842351246562398, 'path_smooth': 8.208042624050156}

Iteration No: 42 ended. Search finished for the next optimal point.
Time taken: 1.4916
Function value obtained: 0.7343
Current minimum: 0.3838
Iteration No: 43 started. Searching for the next optimal point.
Testing parameters: dict_values([0.030911580163750032, 0.14093135946428612, 0.7376724383787223, 0.4208042307969432, 0.3139815112741125, 0.363440801080616, 0.7822152705182588, 0.9825622899067212])
-0.671256392795112 0.9272078353022584

	<<<<< cv = -0.671256392795112   |   corr = 0.9272078353022584 |  score = 0.42271417852369414>>>>>

	{'learning_rate': 0

...

-0.6697916072415063 0.9270507898966472

	<<<<< cv = -0.6697916072415063   |   corr = 0.9270507898966472 |  score = 0.4241869945968758>>>>>

	{'learning_rate': 0.030106720287026812, 'min_child_weight': 1.293069880347309, 'subsample': 0.47008041237405507, 'reg_alpha': 0.7817721707754153, 'reg_lambda': 0.3012469914167221, 'colsample_bynode': 0.28949212754109016, 'colsample_bytree': 0.45920187894232345, 'path_smooth': 9.018425251992081}

Iteration No: 962 ended. Search finished for the next optimal point.
Time taken: 2.1645
Function value obtained: 0.4242
Current minimum: 0.3224
Iteration No: 963 started. Searching for the next optimal point.
Testing parameters: dict_values([0.009189000617466383, 1.7424851237158392, 0.8650237397163468, 0.9595908052784698, 0.8208170716278193, 0.97551930801777, 0.5342131828652507, 0.5048236495605142])
-0.6653820495336034 0.910478285948133

	<<<<< cv = -0.6653820495336034   |   corr = 0.910478285948133 |  score = 0.5698270319441379>>>>>

	{'learning_rate': 0.

-0.6697706877003378 0.9306839937919784

	<<<<< cv = -0.6697706877003378   |   corr = 0.9306839937919784 |  score = 0.39043758250218197>>>>>

	{'learning_rate': 0.031846442934222216, 'min_child_weight': 0.3229620214024861, 'subsample': 0.44316386857711754, 'reg_alpha': 0.9933981308731193, 'reg_lambda': 0.43018274850468385, 'colsample_bynode': 0.30288956578619114, 'colsample_bytree': 0.5057642078046636, 'path_smooth': 0.5380507859521134}

Iteration No: 972 ended. Search finished for the next optimal point.
Time taken: 1.8769
Function value obtained: 0.3904
Current minimum: 0.3224
Iteration No: 973 started. Searching for the next optimal point.
Testing parameters: dict_values([0.028130123906922492, 0.1751043502327167, 0.5131449868830116, 0.5000168583757528, 0.4983917382819335, 0.31770320389387324, 0.5160296175124801, 6.222215906549839])
-0.6689283458079641 0.9355429086936303

	<<<<< cv = -0.6689283458079641   |   corr = 0.9355429086936303 |  score = 0.3450474460544736>>>>>

	{'learning_ra

-0.6719003046694255 0.9335954268407949

	<<<<< cv = -0.6719003046694255   |   corr = 0.9335954268407949 |  score = 0.36320112341929445>>>>>

	{'learning_rate': 0.0235129868425083, 'min_child_weight': 1.2259204850674306, 'subsample': 0.2523606481539888, 'reg_alpha': 0.5538948339115997, 'reg_lambda': 0.22385452668910494, 'colsample_bynode': 0.31312780640025467, 'colsample_bytree': 0.5080943934823154, 'path_smooth': 3.856613183252765}

Iteration No: 982 ended. Search finished for the next optimal point.
Time taken: 2.1620
Function value obtained: 0.3632
Current minimum: 0.3224
Iteration No: 983 started. Searching for the next optimal point.
Testing parameters: dict_values([0.0253534546158221, 0.16178372759922266, 0.6047840969623256, 0.9990394159839725, 0.028346208812002567, 0.27724765216548436, 0.1249184516114438, 3.006251223162107])
-0.6005378829811433 0.7781058591848699

	<<<<< cv = -0.6005378829811433   |   corr = 0.7781058591848699 |  score = 0.9219044600585109>>>>>

	{'learning_rate'

-0.6150477208313374 0.808129354921957

	<<<<< cv = -0.6150477208313374   |   corr = 0.808129354921957 |  score = 0.9153115771788674>>>>>

	{'learning_rate': 0.024737370466154047, 'min_child_weight': 1.8726469693722996, 'subsample': 0.7293048016669316, 'reg_alpha': 0.6171749201943724, 'reg_lambda': 0.10788915736909768, 'colsample_bynode': 0.025882170493468755, 'colsample_bytree': 0.9502214307587177, 'path_smooth': 5.747220847386931}

Iteration No: 992 ended. Search finished for the next optimal point.
Time taken: 2.1740
Function value obtained: 0.9153
Current minimum: 0.3224
Iteration No: 993 started. Searching for the next optimal point.
Testing parameters: dict_values([0.034632015562142585, 1.6093112755862768, 0.019656259536599666, 0.6101915239903771, 0.06010242124448885, 0.2794617893660036, 0.4961714849673168, 9.754203141196998])
-0.6673956233996274 0.9293831570375011

	<<<<< cv = -0.6673956233996274   |   corr = 0.9293831570375011 |  score = 0.40259645020663465>>>>>

	{'learning_rat

True

In [45]:
load_pickle("data/best_x0.pkl")

[0.07234854009556661,
 0.10311050258987084,
 0.39703575514097145,
 0.8518533874150784,
 0.6058731277317677,
 0.5296917410366301,
 0.05170479667699425,
 0.2551823769930473]

In [32]:
# score = 0.38375749021082844
# params = {
#     "learning_rate": 0.02672332457612252,
#     "min_child_weight": 0.1025748131188396,
#     "subsample": 0.024613105875167525,
#     "reg_alpha": 0.1158348412187599,
#     "reg_lambda": 0.1077223568696855,
#     "colsample_bynode": 0.28976110683140427,
#     "colsample_bytree": 0.9211410597388182,
#     "path_smooth": 9.126636612845722,
#     "threshold": 0.3111140356904714,
# }

# x0 = [x[1] for x in filter(lambda x: x[0] != "threshold", params.items())]
# dump_pickle(x0, "data/best_x0.pkl")
# dump_pickle(score, "data/best_y0.pkl")

# Testes

In [36]:
%%time

PARAMETER_NAMES = load_pickle("data/best_PARAMETER_NAMES.pkl")
THRESHOLD = load_pickle("data/best_threshold.pkl")
x0 = load_pickle("data/best_x0.pkl")
score = load_pickle("data/best_y0.pkl")
params = dict(zip(PARAMETER_NAMES,x0))

par = deepcopy(params)

# threshold = 0.3
threshold = THRESHOLD


if "threshold" in par:
    threshold = par.pop("threshold")

estimator = LGBMClassifier(
    n_jobs=n_jobs,
    random_state=RANDOM_STATE,
    **par,
    **DISC_PAR,
)

print(estimator)

fit_params = {
    "early_stopping_rounds": EARLY_STOPPING_ROUNDS,
    "eval_metric": EVAL_METRIC,
    "verbose": VERBOSE,
}

cv = cross_validate_score(
    X=train.drop(["y","id"], 1,errors="ignore"),
    y=train["y"],
    estimator=estimator,
    fit_params=fit_params,
    n_folds=N_FOLDS,
    scoring=SCORING,
    threshold=threshold,
    random_state=RANDOM_STATE,
    verbose=0,
)
print(cv)

LGBMClassifier(colsample_bynode=0.29135698491277934,
               colsample_bytree=0.5008174465107056,
               learning_rate=0.026523837772534035, max_delta_step=0,
               max_depth=6, min_child_weight=0.6171530028591325,
               n_estimators=200, n_jobs=16, path_smooth=2.9570187759837157,
               random_state=42, reg_alpha=0.8875298293774123,
               reg_lambda=0.5162781474994862, subsample=0.08047602405715061)
-0.6711785007857382
CPU times: user 14.3 s, sys: 39.7 ms, total: 14.3 s
Wall time: 946 ms


In [37]:
# Separando os dados de treinamento para essa fold
train_data = train.copy()

# Separando os dados de teste para esse fold
test_data = test.copy()

X_train = train_data.drop(columns=["fold", "y","id"],errors='ignore').values

X_test = test_data.drop(columns=["fold", "y","id"],errors='ignore').values

y_train = train_data["y"].values

y_test = TARGET.values

fit_params.pop('eval_set')
fit_params["eval_set"] = [(X_test, y_test)]

estimator.fit(X_train, y_train, **fit_params)

y_pred = predict(estimator, X_test, threshold=threshold).astype(int)

corr = pd.DataFrame({0: TARGET, 1: y_pred}).corr()[0][1]

if pd.isnull(corr):
    corr = 0

print(corr)

score = score_function(cv,corr)
    
#score = 0.5 * (cv / 0.71 + 1) ** 2 + 0.5 * (corr / 0.99 - 1) ** 2

0.9379658862108163


In [28]:
pd.read_csv(sub_path + "submission_41_teste.csv")[["id"]].assign(predicted=y_pred).to_csv(
    sub_path + "submission_47_teste.csv", index=False
)

In [29]:
np.corrcoef(
pd.read_csv(sub_path + "submission_41_teste.csv")['predicted'],
pd.read_csv(sub_path + "submission_47_teste.csv")['predicted'])

array([[1.        , 0.98955426],
       [0.98955426, 1.        ]])